# Analyzing nanopore sequencing data for AMBIC

## Installing dependencies

In [ ]:
# using conda environment

Analysis pipeline is written as a snakemake pipeline.

In [9]:
%%python 
print("hi")
#%%writefile nanopore.config
#root: /home/isac/Data/ambic/nih

bash: fg: %%python: no such job
bash: syntax error near unexpected token `"hi"'


: 2

In [7]:
ls

analysis                  nanopore_snakemake.ipynb  snakemake
atacseq_sample_info.csv   README.md                 snakemake_config.yml
dag                       scripts                   snakemake_old
nanopore_sample_info.csv  Snakefile
